In [ ]:
epoch_size=500
batch=16
random_seed=1
split_ratio=0.3
var_name="*your_target_y_value_column_name*"
constraint=0 #for remove outlier
random_seed=531

from tensorflow import keras
from keras.layers.core import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, Input, GlobalMaxPooling2D, Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, AveragePooling2D
from keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
from keras.initializers import glorot_uniform
from matplotlib import pyplot as plt
import numpy as np
import argparse
import locale
import os
import pandas as pd
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)
os.environ['PYTHONHASHSEED'] = str(42)

inputPath = "*your_y_value_file*" 

#ex) csv file format, column : data_no x1 x2 x3

df = pd.read_csv(inputPath, sep="\t", header=0)
for i in df.index:
  if df[var_name][i]<constraint:
    df.drop([i], inplace=True)

In [ ]:
image_path="*YOUR IMAGE PATH*"

a=0
file_list=os.listdir(image_path)
image_data=[]
for i in range(len(df['data_no'])):
    image_num=df['data_no'][i]
    image_name="*your_image_name_format_with_data_no*".format(image_num)
    #ex) image_name="{}.jpg",format(image_num)
    matching = [s for s in file_list if image_name in s] 
    if len(matching)!=0:
        file_name=matching[0]
        base_path=os.path.join(image_path,file_name)
        train_array = cv2.imread(base_path, cv2.IMREAD_COLOR)
        train_array = cv2.resize(train_array, dsize=(224,224))
        image_data.append(train_array)
        a=a+1
        if a%100 ==0:
            print(a)
print(a)
print("complete image load")

In [ ]:
from sklearn.model_selection import train_test_split

max_var=max(df[var_name])
y_data=df[var_name]
y_data=y_data/max_var

split = train_test_split(y_data, image_data, test_size=split_ratio, random_state=random_seed)
(y_train,y_valid,image_train,image_valid)=split

x_train=np.array(image_train)/255
x_valid=np.array(image_valid)/255

input_shape = x_train.shape[1:]

In [ ]:
def identity_block(X, f, filters, stage, block):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters
    X_shortcut = X
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    return X

In [ ]:
def convolutional_block(X, f, filters, stage, block, s=2):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters
    X_shortcut = X
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    return X

In [ ]:
def ResNet50(input_shape=input_shape):
    X_input = Input(input_shape)
    X = ZeroPadding2D((1, 1))(X_input)
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')
    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')
    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    X = Flatten()(X)
    X = Dense(1, activation='linear', kernel_initializer=glorot_uniform(seed=0))(X)
    model = Model(inputs=X_input, outputs=X, name='ResNet50')
    return model 

In [ ]:
clear_session()
model = ResNet50(input_shape)
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)
history=model.fit(x_train,y_train, epochs=epoch_size, batch_size=batch, shuffle=True, validation_data=(x_valid,y_valid))

In [ ]:
y_vloss = history.history['val_loss']
y_loss = history.history['loss']
x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c='red', label="Validation-set Loss")
plt.plot(x_len, y_loss, marker='.', c='blue', label="Train-set Loss")
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
preds = model.predict(x_valid)
pred_val=preds.flatten()*max_var
testY=np.asarray(y_valid)
real_val=testY*max_var
print("%s"%(var_name))
print("real \t prediction \t abs_error")
for i in range(len(testY)):
  print("{:.2f} \t {:.2f} \t {:.2f}".format(real_val[i], pred_val[i],abs(real_val[i]-pred_val[i])))

In [ ]:
max_v=max(max(real_val),max(pred_val))
min_v=min(min(real_val),min(pred_val))
plt.rcParams["figure.figsize"] = (5,5)
plt.rcParams['axes.grid'] = True 
plt.scatter(real_val,pred_val)
plt.plot([min_v,max_v],[min_v,max_v],'-k')
plt.xlabel("real %s"%(var_name))
plt.ylabel("prediction %s"%(var_name))
plt.show()